# Reporte técnico: Trabajo 01

Integrantes:

- Javier de Jesús Silva

- Johan Madroñero Cuervo

- Dioselin Esteban Brito

- Santiago Salazar Ramirez

# Desarrollo

## Parte 1: Optimización numérica

### Considere las siguientes funciones de prueba

- Función de Rosenbrock:
$$
{\displaystyle f(\mathbf {x} )=\sum _{i=1}^{N-1}[100(x_{i+1}-x_{i}^{2})^{2}+(1-x_{i})^{2}]\quad {donde}\quad \mathbf {x} =(x_{1},\ldots ,x_{N})\in \mathbb {R} ^{N}}
$$

- Función de Rastrigin:
$$
f(\mathbf {x} )=An+\sum _{i=1}^{n}\left[x_{i}^{2}-A\cos(2\pi x_{i})\right]
$$

- Función de Schwefel
$$
f(\mathbf {x} ) = 418.9829d - \sum_{i=1}^{d}{x_{i} \sin{\sqrt{\lvert x_{i} \rvert}}}
$$

- Función de Griewank
$$
f_n(x_1,...,x_n)=1+\frac{1}{4000}\sum_{i=1}^{n}x_{i}^{2}-\prod_{i=1}^{n}\cos{\frac{x_i}{\sqrt{i}}}
$$

- Función de Goldstein-Price
$$
f(\mathbf{x})=\left[1+\left(x_1+x_2+1\right)^2\left(19-14 x_1+3 x_1^2-14 x_2+6 x_1 x_2+3 x_2^2\right)\right] \times
$$

$$
\left[30+\left(2 x_1-3 x_2\right)^2\left(18-32 x_1+12 x_1^2+48 x_2-36 x_1 x_2+27 x_2^2\right)\right]
$$

- Función Six-Hump Camel

$$
f(\mathbf{x})=\left(4-2.1 x_1^2+\frac{x_1^4}{3}\right) x_1^2+x_1 x_2+\left(-4+4 x_2^2\right) x_2^2
$$


### Escoja dos funciones de prueba

Las funciones escogidas fueron  **Rastrigin** y **Schwefel**. Las cuales definimos a continuación:

#### Definición 1.1 (Rastrigin)
En un dominio de n dimensiones está definido por:

$$
f(\mathbf {x} )=An+\sum _{i=1}^{n}\left[x_{i}^{2}-A\cos(2\pi x_{i})\right]
$$

Donde $$ A = 10 $$ y $$ x_{i} \in[-5.12, 5.12]$$

- El mínimo global se encuentra en $$ \textbf{x = 0} $$ donde $$ f(\mathbf {x}) = \mathbf{0} $$
- El máximo valor para $$ x_{i} \in [-5.12, 5.12 ] $$ se encuentra en $$ x_{i} \in [\pm4.52299366...,...,\pm4.52299366...]$$

#### Definición 1.2 (Schwefel)
Para d dimensiones está definido por:

$$
f(\mathbf {x} ) = 418.9829d - \sum_{i=1}^{d}{x_{i} \sin{\sqrt{\lvert x_{i} \rvert}}}
$$

- La función típicamente es evaluada en el hipercubo $$ x_{i} \in[-500, 500],$$ para $$i = 1,..., d $$
- El mínimo global se encuentra en $$ \textbf{x} = (420.9687,...,420.9687) $$ donde $$ f(\mathbf {x}) = \mathbf{0} $$


In [ ]:
# Instalación de las librerias

!apt update -y
!apt install ffmpeg -y
!pip install ipywidgets==8.0.4 
!pip install pygad

In [ ]:
# Importamos librerarías necesarias 

import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
import pygad
from matplotlib.animation import FuncAnimation
from scipy.optimize import differential_evolution
import warnings
warnings.filterwarnings("ignore")

## Método de descenso por gradiente

### Rastrigin

Utilizamos las funciones de Rastrigin en dos y tres dimensiones para esta demostración del descenso de gradiente. Para graficar dichas funciones utilizamos las librerías matplotlib y numpy, para crear cada video se graficaba el punto que corresponde a la enésima iteración del descenso, y se guarda la imagen de dicho punto sobre la gráfica como uno o tres fotogramas del video, dependiendo de qué tan rápido descienda la función. Inicialmente, para la función de Rastrigin el descenso por gradiente convergía a un mínimo local que no correspondía a la sección de la curva en la que el punto inicial empezaba, esto se debía a que la tasa de aprendizaje (learning rate) era muy grande, lo que provocaba que en ciertas iteraciones el punto "saltara" desde un mínimo a otro de la curva, es decir, la multiplicación learning_rate*gradiente podía resultar en ciertas ocasiones (especialmente en puntos a mitad de camino entre los mínimos y máximos, donde el valor del gradiente es muy alto) en un valor muy grande, haciendo que el desplazamiento en la coordenada X fuese excesivo.

Video 1

Descenso de gradiente para la función de Rastrigin en dos dimensiones.

In [ ]:
# Definimos la funcion de perdida o costo (loss/cost function)
def rastrigin_function_2D(theta):
    func = 10 + theta[0]**2 - 10*math.cos(2*math.pi*theta[0])
    return func

def rastrigin_function_3D(theta):
    func = 20 + theta[0]**2 - 10*math.cos(2*math.pi*theta[0]) + theta[1]**2 - 10*math.cos(2*math.pi*theta[1])
    return func

# Definimos el la funcion gradiente para cada una de estas funciones.
def rastrigin_function_2D_grad(theta):
    grad = np.zeros(1)
    grad[0] = 2*theta[0] + 2*math.pi*10*math.sin(2*math.pi*theta[0])
    return grad

def rastrigin_function_3D_grad(theta):
    grad = np.zeros(2)
    grad[0] = 2*theta[0] + 2*math.pi*10*math.sin(2*math.pi*theta[0])
    grad[1] = 2*theta[1] + 2*math.pi*10*math.sin(2*math.pi*theta[1])
    return grad

# Set the learning rate and number of iterations
alpha = 0.0001
iterations = 10000

# Initialize the parameter vector
theta = np.random.uniform(low=-1, high=1.0, size=(1)) * 5.12
theta_2D = np.random.uniform(low=-1, high=1.0, size=(2)) * 5.12

rastrigin_2d_points = []
rastrigin_3d_points = []

x_i = np.linspace(-5.12, 5.12, 200)
y_2d = 10 + np.power(x_i, 2) - 10*np.cos(2*math.pi*x_i)
y_i = np.linspace(-5.12, 5.12, 100)
plt.ioff()
fig, ax = plt.subplots()

# Perform gradient descent 
for i in range(iterations):
    rastrigin_2d_points.append([theta[0], rastrigin_function_2D(theta)])
    grad = rastrigin_function_2D_grad(theta)
    theta = theta - alpha * grad

for i in range(iterations):
    rastrigin_3d_points.append([theta_2D[0], theta_2D[1]])
    grad = rastrigin_function_3D_grad(theta_2D)
    theta_2D = theta_2D - alpha * grad

FPS = 20
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "descent_rastrigin"
video_title_2 = "descent_rastrigin_3D"
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, 100):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(rastrigin_2d_points[i][0], rastrigin_2d_points[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        writerObj.grab_frame()
        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video
Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)



Como podemos evidenciar en el Video 1, el punto rojo (inicialmente al azar) empieza a descender hacia el mínimo local de la sección de la curva en la que se encuentra, en este caso no hay criterio de parada, sino que el proceso se detiene después de 10000 iteraciones, las cuales observamos, eran más que suficientes para que cualquier punto pudiera converger a su mínimo local correspondiente. Las únicas excepciones a esto son los casos donde el punto aparece exactamente en un máximo (gradiente igual al vector 0), donde no habría desplazamiento hacia ninguna dirección, quedándose el punto estático en dichos lugares.

### Rastrigin 3D animation

Para la función de Rastrigin en el Video 2 (y Schwefel) en tres dimensiones, utilizamos las llamadas líneas de contorno o curvas de nivel, donde podemos ver desde una perspectiva "aérea" o "top-down" como el punto converge a un mínimo local. Tanto en 3D como en 2D se utiliza un learning rate de 0.001, con el propósito de evitar saltos indeseados del punto, de un mínimo a otro.

Video 2

Descenso de gradiente para la función de Rastrigin en tres dimensiones.

In [ ]:
X, Y = np.meshgrid(x_i, y_i)
Z = 20 + np.power(X, 2) - 10*np.cos(2*math.pi*X) + np.power(Y, 2) - 10*np.cos(2*math.pi*Y)

with writerObj.saving(fig, video_title_2+".mp4", dpi):
    for i in range(0, 200):
        ax.contour(X, Y, Z)
        ax.scatter(rastrigin_3d_points[i][0], rastrigin_3d_points[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        writerObj.grab_frame()
        writerObj.grab_frame()
        ax.cla()

Video("/work/" + video_title_2 + ".mp4", embed=True, width=640, height=480)

### Schwefel

La función de Schwefel, a su vez, también converge en ≤10000 iteraciones, pero nótese de que se utilizan para el Video 3 las 5000 primeras iteraciones usando saltos de 10 en 10, esto se debe a que la función de Schwefel converge más lento que la función de Rastrigin, donde usando tan solo las 200 primeras iteraciones la función convergía sin importar el punto donde empezara el proceso iterativo. Cabe resaltar que aunque el learning rate que se escogió para la función de Schwefel fuera más grande (0.0075 vs 0.001), esta función aún converge más lentamente que la función de Rastrigin. Esto se debe a que el gradiente de esta función es mucho menor (la función es menos inclinada), y a que el dominio de la función es mayor, por lo que se tiene que recorrer más distancia para función a un mínimo.

Video 3

Descenso de gradiente para la función de Schwefel en dos dimensiones.

In [ ]:
# Define the cost function
def schwefel_function_2D(theta):
    # Implement your cost function here
    func = 418.9829 - theta[0]*math.sin(math.sqrt(abs(theta[0])))
    return func

def schwefel_function_3D(theta):
    # Implement your cost function here
    func = 418.9829*2 - theta[0]*math.sin(math.sqrt(abs(theta[0]))) - theta[1]*math.sin(math.sqrt(abs(theta[1])))
    return func

# Define the gradient function
def schwefel_function_2D_grad(theta):
    grad = np.zeros(1)
    side = 1
    if theta[0] < 0:
        side = -1
    grad[0] = -math.sin(math.sqrt(abs(theta[0]))) - theta[0]*math.cos(math.sqrt(abs(theta[0])))*side*(0.5*1/math.sqrt(abs(theta[0])))
    return grad

def schwefel_function_3D_grad(theta):
    grad = np.zeros(2)
    side_t1 = 1
    side_t2 = 1

    if theta[0] < 0:
        side_t1 = -1

    if theta[1] < 0:
        side_t2 = -1

    grad[0] = -math.sin(math.sqrt(abs(theta[0]))) - (theta[0])*math.cos(math.sqrt(abs(theta[0])))*side_t1*(0.5*1/math.sqrt(abs(theta[0])))
    grad[1] = -math.sin(math.sqrt(abs(theta[1]))) - (theta[1])*math.cos(math.sqrt(abs(theta[1])))*side_t2*(0.5*1/math.sqrt(abs(theta[1])))
    return grad

# Set the learning rate and number of iterations
alpha = 0.0075
iterations = 10000

# Initialize the parameter vector
theta_s = np.random.uniform(low=-1, high=1.0, size=(1)) * 500
theta_2D_s = np.random.uniform(low=-1, high=1.0, size=(2)) * 500


plt.ioff()
fig, ax = plt.subplots()
schwefel_2d_points = []
schwefel_3d_points = []

x_i = np.linspace(-500, 500, 200)
y_2d = -x_i*np.sin(np.sqrt(np.abs(x_i))) + 418.9829
y_i = np.linspace(-500, 500, 100)

# Perform gradient descent 
for i in range(iterations):
    #cost = rastrigin_function_2D(theta_s)
    #print(cost)
    schwefel_2d_points.append([theta_s[0], schwefel_function_2D(theta_s)])
    grad = schwefel_function_2D_grad(theta_s)
    #print(grad)
    theta_s = theta_s - alpha * grad



for i in range(iterations):
    #cost = rastrigin_function_3D(theta_2D_s)
    schwefel_3d_points.append([theta_2D_s[0], theta_2D_s[1]])
    grad = schwefel_function_3D_grad(theta_2D_s)
    theta_2D_s = theta_2D_s - alpha * grad



dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "descent_schwefel"
video_title_2 = "descent_schwefel_3D"

with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, 5000, 10):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(schwefel_2d_points[i][0], schwefel_2d_points[i][1], c="red")
        fig.canvas.draw()
        
        writerObj.grab_frame()

        ax.cla()

from IPython.display import Video

Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)

### Schwefel 3D animation

Como dato final, nótese que para las 2 funciones en sus versiones de dos y tres dimensiones, los gradientes son idénticos, lo que implica que para valores iniciales iguales, el recorrido será el mismo tanto en 2 como en 3 dimensiones.

Esto se debe a que en ambas funciones los términos que se agregan para aumentar la dimensión son a su vez funciones de (únicamente) la nueva variable Xn que se agrega, por lo que las derivadas parciales con respecto cierta variable son siempre las mismas, sin importar cuantos términos nuevos se sumen. En el Vídeo 4 se visualiza la ejecución del algoritmo

Video 4

Descenso de gradiente para la función de Schwefel en tres dimensiones.

In [ ]:
X, Y = np.meshgrid(x_i, y_i)
Z = 418.9829*2 - X*np.sin(np.sqrt(np.abs(X))) - Y*np.sin(np.sqrt(np.abs(Y)))

with writerObj.saving(fig, video_title_2+".mp4", dpi):
    for i in range(0, 5000, 10):
        ax.contour(X, Y, Z)
        ax.scatter(schwefel_3d_points[i][0], schwefel_3d_points[i][1], c="red")
        fig.canvas.draw()
        
        writerObj.grab_frame()

        ax.cla()

Video("/work/" + video_title_2 + ".mp4", embed=True, width=640, height=480)

## Algoritmos Evolutivos

En esta sección nos enfocaremos en optimizar las funciones escogidas usando la librería **pygad**.

### Rastrigin 2D

Para la función **Rastrigin** en dos dimensiones permititmos 50 generaciones. Fue sorprendente encontrar que para una muestra de 10 ejecuciones, el $100\%$ de las ejecuciones se acercó al mínimo global con un error menor o igual a $0.0005\%$.

**Vídeo 5**

*Optimización de la función Rastrigin en dos dimensiones usando **algoritmos evolutivos***

In [ ]:
# Definimos la función fitness
def rastrigin2D(solution, solution_idx):
  fitness = 10 + math.pow(solution[0],2) - 10 * math.cos(2*math.pi*solution[0])
  return(-fitness)

# Definimos la función Rastrigin para graficar más adelante
def rastriginFunction(x):
    y = 10 + math.pow(x, 2) - 10*math.cos(2*math.pi*x)
    return y


ga_instance_rastrigin2D = pygad.GA(num_generations=50,
                          num_parents_mating=2,
                          fitness_func=rastrigin2D,
                          sol_per_pop=10,
                          num_genes=1,
                          init_range_low=-5.12,
                          init_range_high=5.12,
                          parent_selection_type="sss",
                          keep_parents=1,
                          crossover_type="single_point",
                          mutation_type="random",
                          mutation_percent_genes=10,
                          save_solutions=True,
                          random_seed=13)

# Preparamos una lista para guardar las muestras 
muestra = []

# Hacemos 10 ejecuciones y guardamos la mejor solución
for i in range(10):
    ga_instance_rastrigin2D.run()
    muestra.append(ga_instance_rastrigin2D.best_solution()[0])

# Imprimimos las conclusiones
muestra = list(filter(lambda x : abs(x) < 1, muestra))
print(f"Muestras cercanas: {len(muestra)/10 * 100}%")
maximo_muestra = max(muestra)[0] + 1
print(f"Máximo error de la muestra {abs(maximo_muestra - 1)/1}%")

# Preparamos variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()

x_i = np.linspace(-5.12, 5.12, 200)
y_2d = 10 + np.power(x_i, 2) - 10*np.cos(2*math.pi*x_i)
y_i = np.linspace(-5.12, 5.12, 100)

# Guardamos las soluciones que usaremos para el vídeo
solutions = ga_instance_rastrigin2D.solutions
solutions += [list(ga_instance_rastrigin2D.best_solution()[0])]

# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "descent_rastrigin"

# Creamos el vídeo
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, len(solutions)):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(solutions[i][0], rastriginFunction(solutions[i][0]), c="red")
        fig.canvas.draw()
        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)

Podemos observar un comportamiento curioso en el **Vídeo 5**, pues el algoritmo evolutivo nunca presenta preferencia por el mínimo global. En las últimas generaciones se mueve en el intervalo $[-2, 2]$. Este comportamiento lo acuñamos al pequeño dominio en el que está definida la función **Rastrigin** $[-5.12, 5.12]$.

### Rastrigin 3D

Para la función **Rastrigin** en tres dimensiones permititmos 50 generaciones. Nuevamente fue sorprendente encontrar que para una muestra de 10 ejecuciones, el $100\%$ de las ejecuciones se acercó al *mínimo global* con un error menor o igual a $0.001\%$ para la variable independiente **x** y un error menor o igual a $0.001\%$ para la variable independiente **y**.

**Vídeo 6**

*Optimización de la función Rastrigin en tres dimensiones usando **algoritmos evolutivos***

In [ ]:
# Definimos la función fitness
def rastrigin3D(solution, solution_idx):
  factor1 = math.pow(solution[0],2) - (10 * math.cos(2*math.pi*solution[0]))
  factor2 = math.pow(solution[1],2) - (10 * math.cos(2*math.pi*solution[1]))
  fitness = 20 + factor1 + factor2
  return(-fitness)

ga_instance_rastrigin3D = pygad.GA(num_generations=50,
                          num_parents_mating=2,
                          fitness_func=rastrigin3D,
                          sol_per_pop=10,
                          num_genes=2,
                          init_range_low=-5.12,
                          init_range_high=5.12,
                          parent_selection_type="sss",
                          keep_parents=1,
                          crossover_type="single_point",
                          mutation_type="random",
                          mutation_percent_genes=10,
                          save_solutions=True,
                          random_seed=13)

# Preparamos las listas para guardar las muestras
muestra_x = []
muestra_y = []

# Hacemos 10 ejecuciones y guardamos la mejor solución
for i in range(10):
    ga_instance_rastrigin3D.run()
    muestra_x.append(ga_instance_rastrigin3D.best_solution()[0][0])
    muestra_y.append(ga_instance_rastrigin3D.best_solution()[0][1])

# Imprimimos las conclusiones variable x
muestra_x = list(filter(lambda x : abs(x) < 1, muestra_x))
print(f"Muestras variable x cercanas: {len(muestra_x)/10 * 100}%")
maximo_muestra_x = max(muestra_x) + 1
print(f"Máximo error de la muestra variable aleatoria x {abs(maximo_muestra_x - 1)/1}%")

# Imprimimos las conclusiones variable y
muestra_y = list(filter(lambda y : abs(y) < 1, muestra_y))
print(f"Muestras variable y cercanas: {len(muestra_y)/10 * 100}%")
maximo_muestra_y = max(muestra_y) + 1
print(f"Máximo error de la muestra variable aleatoria y {abs(maximo_muestra_y - 1)/1}%")

# Preparamos variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()

solutions = ga_instance_rastrigin3D.solutions
solutions += [list(ga_instance_rastrigin3D.best_solution()[0])]

x_i = np.linspace(-5.12, 5.12, 200)
y_2d = 10 + np.power(x_i, 2) - 10*np.cos(2*math.pi*x_i)
y_i = np.linspace(-5.12, 5.12, 100)

video_title_2 = "Rastrigin 3D"

FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)

# Creamos funciones para graficar en el vídeo
X, Y = np.meshgrid(x_i, y_i)
Z = 20 + np.power(X, 2) - 10*np.cos(2*math.pi*X) + np.power(Y, 2) - 10*np.cos(2*math.pi*Y)

# Creamos el vídeo
with writerObj.saving(fig, video_title_2+".mp4", dpi):
    for i in range(0, len(solutions)):
        ax.contour(X, Y, Z)
        ax.scatter(solutions[i][0], solutions[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title_2 + ".mp4", embed=True, width=640, height=480)

Nuevamente observamos el mismo comportamiento. En el **Vídeo 6** el algoritmo evolutivo nunca presenta preferencia por el mínimo global. En las últimas generaciones se mueve en el intervalo $[-2, 2]$. Este comportamiento lo acuñamos al pequeño dominio en el que está definida la función **Rastrigin** $[-5.12, 5.12]$.

### Schwefel 2D

Para la función **Schwefel** en dos dimensiones permititmos 50 generaciones. Encontramos resultados similares a la función **Rastrigin** que para una muestra de 10 ejecuciones, el $100\%$ de las ejecuciones se acercó al mínimo global con un error menor a $0.000001\%$.

**Vídeo 7**

*Optimización de la función Schwefel en dos dimensiones usando **algoritmos evolutivos***

In [ ]:
# Definimos la función fitness
def schwefel2D(solution, solution_idx):
  factor1 = solution[0] * math.sin(math.pow(abs(solution[0]),0.5))
  fitness = 418.9829 - factor1
  return(-fitness)

ga_instance_schwefel2D = pygad.GA(num_generations=50,
                          num_parents_mating=2,
                          fitness_func=schwefel2D,
                          sol_per_pop=10,
                          num_genes=1,
                          init_range_low=-500,
                          init_range_high=500,
                          parent_selection_type="sss",
                          keep_parents=1,
                          crossover_type="single_point",
                          mutation_type="random",
                          mutation_percent_genes=10,
                          save_solutions=True,
                          random_seed=13)

ga_instance_schwefel2D.run()

# Preparamos una lista para guardar las muestras 
muestra = []

# Hacemos 10 ejecuciones y guardamos la mejor solución
for i in range(10):
    ga_instance_schwefel2D.run()
    muestra.append(ga_instance_schwefel2D.best_solution()[0])

# Imprimimos las conclusiones
muestra = list(map(lambda x : list(x)[0], muestra))
muestra = list(filter(lambda x : abs(abs(x) - 420.9687) < 1, muestra))
print(f"muestras cercanas: {len(muestra)/10 * 100}%")

if muestra:
  maximo_muestra = max(muestra)
  print(f"Máximo error de la muestra {abs(maximo_muestra - 420.9687)/420.9687}%")

# Preparamos las variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()
solutions = ga_instance_schwefel2D.solutions
solutions += [list(ga_instance_schwefel2D.best_solution()[0])]

x_i = np.linspace(-500, 500, 200)
y_2d = -x_i*np.sin(np.sqrt(np.abs(x_i))) + 418.9829
y_i = np.linspace(-500, 500, 100)

# Definimos la función Schwefel para graficar en el vídeo
def schwefelFunction(x):
    y = -x*math.sin(math.sqrt(np.abs(x))) + 418.9829
    return y

FPS = 10
dpi = 100 
writerObj = FFMpegWriter(fps=FPS)
video_title = "descent_schwefel"

# Creamos el vídeo
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, len(solutions)):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(solutions[i][0], schwefelFunction(solutions[i][0]), c="red")
        fig.canvas.draw()
        
        writerObj.grab_frame()

        ax.cla()

from IPython.display import Video

Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)

Podemos observar un comportamiento diferente al mostrado en la función **Rastrigin** en el **Vídeo 7**, pues el algoritmo evolutivo presenta un comportamiento similar al **descenso por gradiente** cerca de un mínimo en las últimas generaciones. Este comportamiento lo acuñamos al amplio dominio en el que está definida la función **Schwefel** $[-500, 500]$.

### Schwefel 3D

Para la función **Schwefel** en tres dimensiones permititmos 50 generaciones. Encontramos un declive en la *exactitud* de los **algoritmos evolutivos**, para una muestra de 10 ejecuciones, el $80\%$ de las ejecuciones se acercó al mínimo global con un error menor o igual a $0.0001\%$ para la variable independiente **x** y un error menor o igual a $0.000004\%$ para la variable independiente **y**.

**Vídeo 8**

*Optimización de la función Schwefel en tres dimensiones usando **algoritmos evolutivos***

In [ ]:
# Definimos la función fitness
def schwefel3D(solution, solution_idx):
  factor1 = solution[0] * math.sin(math.pow(abs(solution[0]),0.5))
  factor2 = solution[1] * math.sin(math.pow(abs(solution[1]),0.5))
  fitness = (418.9829 * 2) - (factor1 + factor2)
  return(-fitness)

ga_instance_schwefel3D = pygad.GA(num_generations=50,
                          num_parents_mating=2,
                          fitness_func=schwefel3D,
                          sol_per_pop=10,
                          num_genes=2,
                          init_range_low=-500,
                          init_range_high=500,
                          parent_selection_type="sss",
                          keep_parents=1,
                          crossover_type="single_point",
                          mutation_type="random",
                          mutation_percent_genes=10,
                          save_solutions=True,
                          random_seed = 15)

# Preparamos las lsitas para guardar las muestras
muestra_x = []
muestra_y = []

# Hacemos 10 ejecuciones y guardamos la mejor solución
for i in range(10):
    ga_instance_schwefel3D.run()
    muestra_x.append(ga_instance_schwefel3D.best_solution()[0][0])
    muestra_y.append(ga_instance_schwefel3D.best_solution()[0][1])

# Imprimimos las conclusiones variable x
muestra_x = list(filter(lambda x : abs(abs(x) - 420.9687) < 1, muestra_x))
print(f"Muestras variable x cercanas: {len(muestra_x)/10 * 100}%")
if muestra_x:
  maximo_muestra_x = max(muestra_x)
  print(f"Máximo error de la muestra variable aleatoria x {abs(maximo_muestra_x - 420.9687)/420.9687}%")

# Imprimimos las conclusiones variable y
muestra_y = list(filter(lambda y : abs(abs(y) - 420.9687) < 1, muestra_y))
print(f"Muestras variable y cercanas: {len(muestra_y)/10 * 100}%")

if muestra_y:
  maximo_muestra_y = max(muestra_y)
  print(f"Máximo error de la muestra variable aleatoria y {abs(maximo_muestra_y - 420.9687)/420.9687}%")

# Preparamos las variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()
fig, ax = plt.subplots()

solutions = ga_instance_schwefel3D.solutions
solutions += [list(ga_instance_schwefel3D.best_solution()[0])]

# Definimos parametros para graficar
x_i = np.linspace(-500, 500, 200)
y_2d = -x_i*np.sin(np.sqrt(np.abs(x_i))) + 418.9829
y_i = np.linspace(-500, 500, 100)

video_title_2 = "Schwefel 3D"

FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)

# Definimos la función Schwefel para ayudarnos a graficar
X, Y = np.meshgrid(x_i, y_i)
Z = 418.9829*2 - X*np.sin(np.sqrt(np.abs(X))) - Y*np.sin(np.sqrt(np.abs(Y)))

#Creamos el vídeo
with writerObj.saving(fig, video_title_2+".mp4", dpi):
    for i in range(0, len(solutions)):
        ax.contour(X, Y, Z)

        ax.scatter(solutions[i][0], solutions[i][1], c="red")

        fig.canvas.draw()

        writerObj.grab_frame()

        ax.cla()

from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title_2 + ".mp4", embed=True, width=640, height=480)

Nuevamente podemos observar un comportamiento diferente al mostrado en la función **Rastrigin** en el **Vídeo 8**, pues el algoritmo evolutivo presenta un comportamiento similar al **descenso por gradiente** cerca de un mínimo en las últimas generaciones. Este comportamiento lo acuñamos al amplio dominio en el que está definida la función **Schwefel** $[-500, 500]$.

## Optimización de partículas

La optimización de partículas es una técnica de optimización computacional que se utiliza para resolver problemas de optimización en los que se busca encontrar el valor mínimo o máximo de una función de varias variables. Esta técnica se basa en la simulación de un grupo de partículas, que representan posibles soluciones del problema de optimización, y su movimiento en un espacio de búsqueda multidimensional. El **Vídeo 9** y **Vídeo 10** ilustran Rastrigin en optimización de partículas (2D y 3D), mientras que el **Vídeo 11** y **Vídeo 12** se ilustran Schwefel 2D y 3D respectivamente.

### Rastrigin 2D

**Vídeo 9**

*Optimización de la función Rastrigin en dos dimensiones usando **Optimización de partículas***

In [ ]:
def rastrigin(x):
    return 10 + np.power(x, 2) - 10*np.cos(2*math.pi*x)

# Definimos la función Rastrigin para graficar más adelante
def rastriginFunction(x):
    y = 10 + math.pow(x, 2) - 10*math.cos(2*math.pi*x)
    return y


def run_PSO(n_particles=500, n_iterations = 200,  omega=0.5, phi_p=0.3, phi_g=0.4):
    """ PSO algorithm to a funcion already defined.
    Params:
        omega = Particle weight (intertial)
        phi_p = Particle best weight
        phi_g = Global global weight
    """
    global x_best_p_global, x_particles, y_particles, u_particles, v_particles

    ## Initialization:
    # Guardamos las soluciones que usaremos para el vídeo
    solutions = []

    # Definición del intervalo de búsqueda
    x_lo = -5.12
    x_up = 5.12

    # Generación de la función de Rastrigin en el intervalo definido
    n_points = 1000
    x = np.linspace(x_lo, x_up, n_points)
    y = rastrigin(x)

    # Inicialización de las posiciones de las partículas
    x_particles = np.zeros((n_particles, n_iterations))
    x_particles[:, 0] = np.random.uniform(x_lo, x_up, size=n_particles)

    # Inicialización de la mejor posición de cada partícula
    x_best_particles = np.copy(x_particles[:, 0])

    # Evaluación de la función objetivo en la posición de cada partícula
    y_particles = rastrigin(x_particles[:, 0])

    # Inicialización de la mejor posición global
    y_best_global = np.min(y_particles[:])
    index_best_global = np.argmin(y_particles[:])
    x_best_p_global = np.copy(x_particles[index_best_global,0])

    # Definición de la unidad de velocidad
    velocity_lo = x_lo-x_up 
    velocity_up = x_up-x_lo 

    # Inicialización de las velocidades de las partículas
    u_particles = np.zeros((n_particles, n_iterations))
    u_particles[:, 0] = 0.1*np.random.uniform(velocity_lo, velocity_up, size=n_particles)

    # Inicialización de v_particles, utilizado para graficar las velocidades
    v_particles = np.zeros((n_particles, n_iterations))

    # Comienza el algoritmo PSO
    iteration = 1
    while iteration <= n_iterations-1:
        solutions.append(x_particles[:, iteration-1])
        for i in range(n_particles):
            # Actualización de la posición y velocidad de cada partícula
            x_p = x_particles[i, iteration-1]
            u_p = u_particles[i, iteration-1]
            x_best_p = x_best_particles[i]
            r_p = np.random.uniform(0, 1)
            r_g = np.random.uniform(0, 1)

            # Cálculo de la nueva velocidad y posición de la partícula
            u_p_new = omega*u_p + phi_p*r_p*(x_best_p-x_p) + phi_g*r_g*(x_best_p_global-x_p)
            x_p_new = x_p + u_p_new

            # Se comprueba que la nueva posición esté dentro del intervalo de búsqueda
            if not x_lo <= x_p_new <= x_up: 
                x_p_new = x_p 
                u_p_new = 0

            # Se actualiza la posición y velocidad de la partícula
            x_particles[i, iteration] = np.copy(x_p_new)
            u_particles[i, iteration] = np.copy(u_p_new)

            # Se obtiene la mejor posición y valor de la función objetivo de la partícula i
            y_p_best = rastrigin(x_best_particles[i])

            # Se obtiene el valor de la función objetivo en la nueva posición de la partícula i
            y_p_new = rastrigin(x_p_new)

            # Si la nueva posición de la partícula i mejora su mejor posición anterior, se actualiza
            if y_p_new < y_p_best:
                x_best_particles[i] = np.copy(x_p_new)

                # Se actualiza la mejor posición global si la nueva posición de la partícula i mejora la mejor posición global actual
                y_p_best_global = rastrigin(x_best_p_global)
                if y_p_new < y_p_best_global:
                    x_best_p_global = x_p_new
            

        iteration = iteration + 1

    # print("Best solution found by PSO: x = {solution}, f(x) = {objective}".format(solution=x_best_p_global, objective=rastrigin(x_best_p_global)))

    return solutions

solutions = run_PSO()

# Preparamos variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()

x_i = np.linspace(-5.12, 5.12, 200)
y_2d = 10 + np.power(x_i, 2) - 10*np.cos(2*math.pi*x_i)
y_i = np.linspace(-5.12, 5.12, 100)



# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "pso_rastrigin"

# Creamos el vídeo
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, len(solutions)):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(solutions[i][0], rastriginFunction(solutions[i][0]), c="red")
        fig.canvas.draw()
        writerObj.grab_frame()
        ax.cla()
        
from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)




### Rastrigin 3D

**Vídeo 10**

*Optimización de la función Rastrigin en dos dimensiones usando **Optimización de partículas***

In [ ]:
def rastringin(v):
    "Rastrigin 3D"
    x, y = v
    factor1 = x**2 - (10 * np.cos(2*math.pi*x))
    factor2 = y**2 - (10 * np.cos(2*math.pi*y))
    return (20 + factor1 + factor2)
    
def run_PSO(n_particles=500, n_iterations = 350, omega=0.5, phi_p=0.6, phi_g=0.6):
    """ PSO algorithm to a funcion already defined.
    Params:
        omega = 0.3  # Particle weight (intertial)
        phi_p = 0.7  # particle best weight
        phi_g = 0.7  # global global weight
    
    """
    global x_best_p_global, y_best_p_global, z_p_best_global, \
           x_particles, y_particles, z_particles, \
           u_particles, v_particles
    
    # Note: we are using global variables to ease the use of interactive widgets
    # This code will work fine without the global (and actually it will be safer)

    lo_b = -5.12 # lower bound
    up_b =  5.12 # upper bound

    limits=([lo_b, up_b], # x bounds
            [lo_b, up_b]) # y bounds

    x_lo = limits[0][0]
    x_up = limits[0][1]
    y_lo = limits[1][0]
    y_up = limits[1][1]

    # Guardamos las soluciones que usaremos para el vídeo
    solutions = []

    # Initialazing x postion of particles
    
    x_particles = np.zeros((n_iterations, n_particles))
    x_particles[0,:] = np.random.uniform(lo_b, up_b, size=n_particles)

    # Initialazing y postion of particles
    y_particles = np.zeros((n_iterations, n_particles))
    y_particles[0,:] = np.random.uniform(lo_b, up_b, size=n_particles)

    # Initialazing best praticles
    x_best_particles = np.copy(x_particles[0,:])
    y_best_particles = np.copy(y_particles[0,:])
    
    # Calculate Objective function (aka fitness function)
    z_particles = np.zeros((n_iterations, n_particles))

    for i in range(n_particles):
        z_particles[0,i] = rastringin((x_particles[0,i],y_particles[0,i]))

    z_best_global = np.min(z_particles[0,:])
    index_best_global = np.argmin(z_particles[0,:])

    x_best_p_global = x_particles[0,index_best_global]
    y_best_p_global = y_particles[0,index_best_global]

    # Initialazin velocity
    velocity_lo = lo_b-up_b  # [L/iteration]
    velocity_up = up_b-lo_b  # [L/iteration] 

    v_max = 0.07 # [L/iteration]

    u_particles = np.zeros((n_iterations, n_particles))
    u_particles[0,:] = 0.1*np.random.uniform(velocity_lo, velocity_up, size=n_particles)

    v_particles = np.zeros((n_iterations, n_particles))
    v_particles[0,:] = 0.1*np.random.uniform(velocity_lo, velocity_up, size=n_particles)

    

    # PSO STARTS
    iteration = 1
    while iteration <= n_iterations-1:
        solutions.append((x_particles[iteration-1,:], y_particles[iteration-1,:]))
        for i in range(n_particles):
            x_p = x_particles[iteration-1, i]
            y_p = y_particles[iteration-1, i]

            u_p = u_particles[iteration-1, i]
            v_p = v_particles[iteration-1, i]

            x_best_p = x_best_particles[i]
            y_best_p = y_best_particles[i]

            r_p = np.random.uniform(0, 1)
            r_g = np.random.uniform(0, 1)

            u_p_new = omega*u_p + \
                        phi_p*r_p*(x_best_p-x_p) + \
                        phi_g*r_g*(x_best_p_global-x_p)

            v_p_new = omega*v_p + \
                        phi_p*r_p*(y_best_p-y_p) + \
                        phi_g*r_g*(y_best_p_global-y_p)

            # # Velocity control
            # while not (-v_max <= u_p_new <= v_max):  
            #     u_p_new = 0.9*u_p_new 
            # while not (-v_max <= u_p_new <= v_max):  
            #     u_p_new = 0.9*u_p_new 

            x_p_new = x_p + u_p_new
            y_p_new = y_p + v_p_new


            # Ignore new position if it's out of the domain
            if not ((lo_b <= x_p_new <= up_b) and (lo_b <= y_p_new <= up_b)): 
                x_p_new = x_p 
                y_p_new = y_p 

            x_particles[iteration, i] = x_p_new
            y_particles[iteration, i] = y_p_new

            u_particles[iteration, i] = u_p_new
            v_particles[iteration, i] = v_p_new

            # Evaluation            
            z_p_new = rastringin((x_p_new,  y_p_new))
            z_p_best = rastringin((x_best_p, y_best_p))
            
            z_particles[iteration, i] = z_p_new

            if z_p_new < z_p_best:
                x_best_particles[i] = x_p_new
                y_best_particles[i] = y_p_new

                z_p_best_global = rastringin([x_best_p_global, y_best_p_global])

                if z_p_new < z_p_best_global:
                    x_best_p_global = x_p_new
                    y_best_p_global = y_p_new

        # end while loop particles
        iteration = iteration + 1
        
    print("Best solution found: ", x_best_p_global, y_best_p_global, z_p_best_global)

    return solutions

solutions = run_PSO()

# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "PSO_rastrigin_3D"

x_i = np.linspace(-5.12, 5.12, 200)
y_i = np.linspace(-5.12, 5.12, 100)
plt.ioff()
fig, ax = plt.subplots()

X, Y = np.meshgrid(x_i, y_i)
Z = 20 + np.power(X, 2) - 10*np.cos(2*math.pi*X) + np.power(Y, 2) - 10*np.cos(2*math.pi*Y)

with writerObj.saving(fig, video_title +".mp4", dpi):
    for i in range(0, len(solutions)):
        ax.contour(X, Y, Z)
        ax.scatter(solutions[i][0], solutions[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        writerObj.grab_frame()
        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video

Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)



### Schwefel 2D

**Vídeo 11**

*Optimización de la función Schwefel en dos dimensiones usando **Optimización de partículas***

In [ ]:
def schwefel(x):
    return -x * np.sin(np.sqrt(np.abs(x))) + 418.9829

# Definimos la función Schwefel para graficar más adelante
def schwefelFunction(x):
    y = -x*math.sin(math.sqrt(np.abs(x))) + 418.9829
    return y

def run_PSO(n_particles=500, n_iterations = 200,  omega=0.5, phi_p=0.3, phi_g=0.4):
    """ PSO algorithm to a funcion already defined.
    Params:
        omega = 0.5  # Particle weight (intertial)
        phi_p = 0.1  # particle best weight
        phi_g = 0.1  # global global weight
    """
    global x_best_p_global, x_particles, y_particles, u_particles, v_particles

    ## Initialization:
    # Guardamos las soluciones que usaremos para el vídeo
    solutions = []

    # Definición del intervalo de búsqueda
    x_lo = -500
    x_up = 500

    # Generación de la función de Schwefel en el intervalo definido
    n_points = 1000
    x = np.linspace(x_lo, x_up, n_points)
    y = schwefel(x)

    # Inicialización de las posiciones de las partículas
    x_particles = np.zeros((n_particles, n_iterations))
    x_particles[:, 0] = np.random.uniform(x_lo, x_up, size=n_particles)

    # Inicialización de la mejor posición de cada partícula
    x_best_particles = np.copy(x_particles[:, 0])

    # Evaluación de la función objetivo en la posición de cada partícula
    y_particles = schwefel(x_particles[:, 0])

    # Inicialización de la mejor posición global
    y_best_global = np.min(y_particles[:])
    index_best_global = np.argmin(y_particles[:])
    x_best_p_global = np.copy(x_particles[index_best_global,0])

    # Definición de la unidad de velocidad
    velocity_lo = x_lo-x_up 
    velocity_up = x_up-x_lo 

    # Inicialización de las velocidades de las partículas
    u_particles = np.zeros((n_particles, n_iterations))
    u_particles[:, 0] = 0.1*np.random.uniform(velocity_lo, velocity_up, size=n_particles)

    # Inicialización de v_particles, utilizado para graficar las velocidades
    v_particles = np.zeros((n_particles, n_iterations))

    # Comienza el algoritmo PSO
    iteration = 1
    while iteration <= n_iterations-1:
        solutions.append(x_particles[:, iteration-1])
        for i in range(n_particles):
            # Actualización de la posición y velocidad de cada partícula
            x_p = x_particles[i, iteration-1]
            u_p = u_particles[i, iteration-1]
            x_best_p = x_best_particles[i]
            r_p = np.random.uniform(0, 1)
            r_g = np.random.uniform(0, 1)

            # Cálculo de la nueva velocidad y posición de la partícula
            u_p_new = omega*u_p + phi_p*r_p*(x_best_p-x_p) + phi_g*r_g*(x_best_p_global-x_p)
            x_p_new = x_p + u_p_new

            # Se comprueba que la nueva posición esté dentro del intervalo de búsqueda
            if not x_lo <= x_p_new <= x_up: 
                x_p_new = x_p 
                u_p_new = 0

            # Se actualiza la posición y velocidad de la partícula
            x_particles[i, iteration] = np.copy(x_p_new)
            u_particles[i, iteration] = np.copy(u_p_new)

            # Se obtiene la mejor posición y valor de la función objetivo de la partícula i
            y_p_best = schwefel(x_best_particles[i])

            # Se obtiene el valor de la función objetivo en la nueva posición de la partícula i
            y_p_new = schwefel(x_p_new)

            # Si la nueva posición de la partícula i mejora su mejor posición anterior, se actualiza
            if y_p_new < y_p_best:
                x_best_particles[i] = np.copy(x_p_new)

                # Se actualiza la mejor posición global si la nueva posición de la partícula i mejora la mejor posición global actual
                y_p_best_global = schwefel(x_best_p_global)
                if y_p_new < y_p_best_global:
                    x_best_p_global = x_p_new
            

        iteration = iteration + 1

    # print("Best solution found by PSO: x = {solution}, f(x) = {objective}".format(solution=x_best_p_global, objective=rastrigin(x_best_p_global)))

    return solutions

solutions = run_PSO()

# Preparamos variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()

x_i = np.linspace(-500, 500, 200)
y_2d = -x_i*np.sin(np.sqrt(np.abs(x_i))) + 418.9829
y_i = np.linspace(-500, 500, 100)



# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "pso_schwefel"

# Creamos el vídeo
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, len(solutions)):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(solutions[i][0], schwefelFunction(solutions[i][0]), c="red")
        fig.canvas.draw()
        writerObj.grab_frame()
        ax.cla()
        
from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)

### Schwefel 3D

**Vídeo 12**

*Optimización de la función Schwefel en tres dimensiones usando **Optimización de partículas***

In [ ]:
def schwefel(v):
    "Schwefel 3D"
    x, y = v
    factor1 = x * np.sin(abs(x)**0.5)
    factor2 = y * np.sin(abs(y)**0.5)
    return ((418.9829*2) - (factor1 + factor2))
     
def run_PSO(n_particles=500, n_iterations = 350, omega=0.5, phi_p=0.6, phi_g=0.6):
    """ PSO algorithm to a funcion already defined.
    Params:
        omega = 0.5  # Particle weight (intertial)
        phi_p = 0.6  # particle best weight
        phi_g = 0.6  # global global weight
    
    """
    global x_best_p_global, y_best_p_global, z_p_best_global, \
           x_particles, y_particles, z_particles, \
           u_particles, v_particles
    
    # Note: we are using global variables to ease the use of interactive widgets
    # This code will work fine without the global (and actually it will be safer)

    lo_b = -500 # lower bound
    up_b =  500 # upper bound

    limits=([lo_b, up_b], # x bounds
            [lo_b, up_b]) # y bounds

    x_lo = limits[0][0]
    x_up = limits[0][1]
    y_lo = limits[1][0]
    y_up = limits[1][1]

    # Guardamos las soluciones que usaremos para el vídeo
    solutions = []

    # Initialazing x postion of particles
    
    x_particles = np.zeros((n_iterations, n_particles))
    x_particles[0,:] = np.random.uniform(lo_b, up_b, size=n_particles)

    # Initialazing y postion of particles
    y_particles = np.zeros((n_iterations, n_particles))
    y_particles[0,:] = np.random.uniform(lo_b, up_b, size=n_particles)

    # Initialazing best praticles
    x_best_particles = np.copy(x_particles[0,:])
    y_best_particles = np.copy(y_particles[0,:])
    
    # Calculate Objective function (aka fitness function)
    z_particles = np.zeros((n_iterations, n_particles))

    for i in range(n_particles):
        z_particles[0,i] = schwefel((x_particles[0,i],y_particles[0,i]))

    z_best_global = np.min(z_particles[0,:])
    index_best_global = np.argmin(z_particles[0,:])

    x_best_p_global = x_particles[0,index_best_global]
    y_best_p_global = y_particles[0,index_best_global]

    # Initialazin velocity
    velocity_lo = lo_b-up_b  # [L/iteration]
    velocity_up = up_b-lo_b  # [L/iteration] 

    v_max = 0.07 # [L/iteration]

    u_particles = np.zeros((n_iterations, n_particles))
    u_particles[0,:] = 0.1*np.random.uniform(velocity_lo, velocity_up, size=n_particles)

    v_particles = np.zeros((n_iterations, n_particles))
    v_particles[0,:] = 0.1*np.random.uniform(velocity_lo, velocity_up, size=n_particles)

    

    # PSO STARTS
    iteration = 1
    while iteration <= n_iterations-1:
        solutions.append((x_particles[iteration-1,:], y_particles[iteration-1,:]))
        for i in range(n_particles):
            x_p = x_particles[iteration-1, i]
            y_p = y_particles[iteration-1, i]

            u_p = u_particles[iteration-1, i]
            v_p = v_particles[iteration-1, i]

            x_best_p = x_best_particles[i]
            y_best_p = y_best_particles[i]

            r_p = np.random.uniform(0, 1)
            r_g = np.random.uniform(0, 1)

            u_p_new = omega*u_p + \
                        phi_p*r_p*(x_best_p-x_p) + \
                        phi_g*r_g*(x_best_p_global-x_p)

            v_p_new = omega*v_p + \
                        phi_p*r_p*(y_best_p-y_p) + \
                        phi_g*r_g*(y_best_p_global-y_p)


            x_p_new = x_p + u_p_new
            y_p_new = y_p + v_p_new


            # Ignore new position if it's out of the domain
            if not ((lo_b <= x_p_new <= up_b) and (lo_b <= y_p_new <= up_b)): 
                x_p_new = x_p 
                y_p_new = y_p 

            x_particles[iteration, i] = x_p_new
            y_particles[iteration, i] = y_p_new

            u_particles[iteration, i] = u_p_new
            v_particles[iteration, i] = v_p_new

            # Evaluation            
            z_p_new = schwefel((x_p_new,  y_p_new))
            z_p_best = schwefel((x_best_p, y_best_p))
            
            z_particles[iteration, i] = z_p_new

            if z_p_new < z_p_best:
                x_best_particles[i] = x_p_new
                y_best_particles[i] = y_p_new

                z_p_best_global = schwefel([x_best_p_global, y_best_p_global])

                if z_p_new < z_p_best_global:
                    x_best_p_global = x_p_new
                    y_best_p_global = y_p_new

        # end while loop particles
        iteration = iteration + 1
        
    print("Best solution found: ", x_best_p_global, y_best_p_global, z_p_best_global)

    return solutions

solutions = run_PSO()

# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "PSO_schwefel_3D"

x_i = np.linspace(-500, 500, 200)
y_i = np.linspace(-500, 500, 100)
plt.ioff()
fig, ax = plt.subplots()

X, Y = np.meshgrid(x_i, y_i)
Z = 418.9829*2 - X*np.sin(np.sqrt(np.abs(X))) - Y*np.sin(np.sqrt(np.abs(Y)))

with writerObj.saving(fig, video_title +".mp4", dpi):
    for i in range(0, len(solutions)):
        ax.contour(X, Y, Z)
        ax.scatter(solutions[i][0], solutions[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        writerObj.grab_frame()
        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video

Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)

## Evolución diferencial

La evolución diferencial (DE, por sus siglas en inglés) es un algoritmo de optimización basado en poblaciones que se utiliza para resolver problemas de optimización global en espacios de alta dimensionalidad. El algoritmo de DE se inspira en el proceso de evolución biológica y consiste en la generación y manipulación de un conjunto de vectores de parámetros que representan posibles soluciones del problema. El **Vídeo 13** y **Vídeo 14** ilustran Rastrigin en evolución diferencial (2D y 3D), el **Vídeo 15** y **Vídeo 16** Schwefel (2D y 3D).



### Rastrigin 2D

**Vídeo 13**

*Optimización de la función Rastringin en dos dimensiones usando **Evolución diferencial***

In [ ]:
def rastrigin(x):
    return 10 + x**2 - 10*math.cos(2*math.pi*x)

def rastriginFunction(x):
    y = 10 + math.pow(x, 2) - 10*math.cos(2*math.pi*x)
    return y

# Definir función para guardar la mejor solución en cada iteración
def callback(xk, convergence):
    population.append(xk)

bounds = [(-5.12, 5.12)]   # límites de los valores de las variables de decisión
pop_size = 500  # tamaño de la población
max_generations = 500  # número máximo de generaciones
mutation_factor = 0.9  # factor de mutación
crossover_prob = 0.9 # probabilidad de cruce

# Inicializar la población aleatoriamente dentro de los límites
population = [np.random.uniform(b[0], b[1], size=pop_size) for b in bounds]

# Ejecutar el algoritmo de Evolución Diferencial y guardar la población en cada iteración
result = differential_evolution(func=rastrigin, bounds=bounds, popsize=pop_size, maxiter=max_generations,
                                mutation=mutation_factor, recombination=crossover_prob, callback=callback)

# best solution
print('Mejor solución', result.x[0])

# Preparamos variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()

x_i = np.linspace(-5.12, 5.12, 200)
y_2d = 10 + np.power(x_i, 2) - 10*np.cos(2*math.pi*x_i)
y_i = np.linspace(-5.12, 5.12, 100)



# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "de_rastrigin2D"

# Creamos el vídeo
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, len(population)):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(population[i][0], rastriginFunction(population[i][0]), c="red")
        fig.canvas.draw()
        writerObj.grab_frame()
        ax.cla()
        
from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)




### Rastrigin 3D

**Vídeo 14**

*Optimización de la función Rastringin en tres dimensiones usando **Evolución diferencial***

In [ ]:
def rastrigin(v):
    x, y = v
    factor1 = x**2 - (10 * np.cos(2*math.pi*x))
    factor2 = y**2 - (10 * np.cos(2*math.pi*y))
    return (20 + factor1 + factor2)


# Definir función para guardar la mejor solución en cada iteración
def callback(xk, convergence):
    population.append(xk)

bounds = [(-5.12, 5.12)] * 2   # límites de los valores de las variables de decisión
pop_size = 500  # tamaño de la población
max_generations = 500  # número máximo de generaciones
mutation_factor = 0.9  # factor de mutación
crossover_prob = 0.9 # probabilidad de cruce

# Inicializar la población aleatoriamente dentro de los límites
population = [np.random.uniform(b[0], b[1], size=2) for b in bounds]


# Ejecutar el algoritmo de Evolución Diferencial y guardar la población en cada iteración
result = differential_evolution(func=rastrigin, bounds=bounds, popsize=pop_size, maxiter=max_generations,
                                mutation=mutation_factor, recombination=crossover_prob, callback=callback)


# best solution
# print(result)
# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "de_rastrigin_3D"

x_i = np.linspace(-5.12, 5.12, 200)
y_i = np.linspace(-5.12, 5.12, 100)
plt.ioff()
fig, ax = plt.subplots()

X, Y = np.meshgrid(x_i, y_i)
Z = 20 + np.power(X, 2) - 10*np.cos(2*math.pi*X) + np.power(Y, 2) - 10*np.cos(2*math.pi*Y)

with writerObj.saving(fig, video_title +".mp4", dpi):
    for i in range(0, len(population)):
        ax.contour(X, Y, Z)
        ax.scatter(population[i][0], population[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        writerObj.grab_frame()
        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video

Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)

### Schwefel 2D

**Vídeo 15**

*Optimización de la función Schwefel en dos dimensiones usando **Evolución diferencial***

In [ ]:
def schwefel(x):
    return -x * np.sin(np.sqrt(np.abs(x))) + 418.9829

# Definimos la función Schwefel para graficar más adelante
def schwefelFunction(x):
    y = -x*math.sin(math.sqrt(np.abs(x))) + 418.9829
    return y

# Definir función para guardar la mejor solución en cada iteración
def callback(xk, convergence):
    population.append(xk)

bounds = [(-500, 500)]   # límites de los valores de las variables de decisión
pop_size = 500  # tamaño de la población
max_generations = 500  # número máximo de generaciones
mutation_factor = 0.9  # factor de mutación
crossover_prob = 0.9 # probabilidad de cruce

# Inicializar la población aleatoriamente dentro de los límites
population = [np.random.uniform(b[0], b[1], size=pop_size) for b in bounds]

# Ejecutar el algoritmo de Evolución Diferencial y guardar la población en cada iteración
result = differential_evolution(func=schwefel, bounds=bounds, popsize=pop_size, maxiter=max_generations,
                                mutation=mutation_factor, recombination=crossover_prob, callback=callback)

# best solution
print('Mejor solución: ', result.x[0])

# Preparamos variables para el vídeo
plt.ioff()
fig, ax = plt.subplots()

x_i = np.linspace(-500, 500, 200)
y_2d = -x_i*np.sin(np.sqrt(np.abs(x_i))) + 418.9829
y_i = np.linspace(-500, 500, 100)



# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "de_schwefel2D"

# Creamos el vídeo
with writerObj.saving(fig, video_title+".mp4", dpi):
    for i in range(0, len(population)):
        line, = ax.plot(x_i, y_2d, c="blue")
        ax.scatter(population[i][0], schwefelFunction(population[i][0]), c="red")
        fig.canvas.draw()
        writerObj.grab_frame()
        ax.cla()
        
from IPython.display import Video

# Enseñamos el vídeo
Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)



### Schwefel 3D

**Vídeo 16**

*Optimización de la función Schwefel en tres dimensiones usando **Evolución diferencial***

In [ ]:
def schwefel(v):
    x, y = v
    return -x * np.sin(np.sqrt(np.abs(x))) - y * np.sin(np.sqrt(np.abs(y))) + 418.9829 * 2

# Definir función para guardar la mejor solución en cada iteración
def callback(xk, convergence):
    population.append(xk)

bounds = [(-500, 500)] * 2   # límites de los valores de las variables de decisión
pop_size = 500  # tamaño de la población
max_generations = 500  # número máximo de generaciones
mutation_factor = 0.9  # factor de mutación
crossover_prob = 0.9 # probabilidad de cruce

# Inicializar la población aleatoriamente dentro de los límites
population = [np.random.uniform(b[0], b[1], size=2) for b in bounds]


# Ejecutar el algoritmo de Evolución Diferencial y guardar la población en cada iteración
result = differential_evolution(func=schwefel, bounds=bounds, popsize=pop_size, maxiter=max_generations,
                                mutation=mutation_factor, recombination=crossover_prob, callback=callback)


# best solution
# print(result)


# Preparamos parametros para el vídeo
FPS = 10
dpi = 100 # quality of the video
writerObj = FFMpegWriter(fps=FPS)
video_title = "de_schwefel_3D"

x_i = np.linspace(-500, 500, 200)
y_i = np.linspace(-500, 500, 100)
plt.ioff()
fig, ax = plt.subplots()

X, Y = np.meshgrid(x_i, y_i)
Z = 418.9829*2 - X*np.sin(np.sqrt(np.abs(X))) - Y*np.sin(np.sqrt(np.abs(Y)))

with writerObj.saving(fig, video_title +".mp4", dpi):
    for i in range(0, len(population)):
        ax.contour(X, Y, Z)
        ax.scatter(population[i][0], population[i][1], c="red")
        fig.canvas.draw()

        writerObj.grab_frame()
        writerObj.grab_frame()
        writerObj.grab_frame()
        ax.cla()

from IPython.display import Video

Video("/work/" + video_title + ".mp4", embed=True, width=640, height=480)


## Conclusiones generales

Los métodos de descenso por gradiente aportaron una forma eficiente y efectiva de encontrar el mínimo local de una función diferenciable. Los métodos de descenso por gradiente se basan en la iteración de un proceso de actualización de la solución utilizando la dirección de máxima disminución de la función en cada iteración. Los métodos de descenso por gradiente son sencillos y efectivos, y se utilizan ampliamente en la optimización de funciones convexas y en problemas de aprendizaje automático, como la regresión lineal y la regresión logística.

Los métodos heurísticos, como los algoritmos evolutivos, el de optimización de partículas, y la evolución diferencial, aportaron una forma alternativa de resolver problemas de optimización complicados en los que los métodos de descenso por gradiente son menos efectivos. Los métodos heurísticos se basan en la simulación de un proceso de búsqueda de soluciones candidatas, en lugar de calcular la dirección exacta del mínimo local de la función. Los métodos heurísticos son capaces de explorar diferentes áreas del espacio de búsqueda y pueden encontrar soluciones óptimas globales en funciones no convexas y en problemas de alta dimensionalidad.

Finalmente, los métodos de descenso por gradiente son más efectivos en funciones convexas y en problemas de baja dimensionalidad, mientras que los métodos heurísticos son más adecuados para problemas de optimización complicados con múltiples mínimos locales y espacios de búsqueda de alta dimensionalidad. 



## Punto 2

In [ ]:
# Instalación de las librerias
!pip install numpy
!pip install matplotlib
!pip install pandas

In [ ]:
# Primer paso: Importar las librerias necesarias 
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from numpy.random import seed
seed(123)

import warnings
warnings.filterwarnings("ignore")

### Coste entre ciudades: ¿Cómo se halló este valor?

El procedimiento consta de los siguientes pasos:

undefined. Encontrar el tiempo promedio que se tarda un automóvil en ir desde la ciudad x1 hasta la ciudad x2, estos datos se pudieron obtener gracias a rome2rio (Rome2rio: descubre cómo llegar a cualquier lugar, s. f.). En la Figura 1 se visualizan los tiempos entre ciudades

Figura 1

Tiempos entre ciudades

In [ ]:
tiemposDistancias = pd.read_csv("TiemposDistancias.csv",sep=',')
tiemposDistancias

Nota: Los tiempos están definidos en horas.

2. Con los tiempos entre ciudades se halla el coste por hora del vendedor, el procedimiento es multiplicar el valor por hora del vendedor por el tiempo entre las dos ciudades. El valor por hora del vendedor se definió en 4833 pesos colombianos, este valor se decidió por el salario mínimo actual (Alvarez, 2022). En la Figura 2 están los costes del pago del vendedor para cada trayecto entre ciudades.

Figura 2

Costes del vendedor

In [ ]:
costesVendedor = pd.read_csv("CostesVendedor.csv",sep=',')
costesVendedor

Nota: Los datos salen de la operación entre el salario del vendedor por hora y el tiempo de recorrido entre ciudades

3. Se hallan las distancias entre las ciudades, esta información se usa en el cuarto paso para hallar el coste de la gasolina para los recorridos entre ciudades. La distancias entre ciudades se obtuvo en peajesencolombia.com (Edward Daniel Vidal garcia - www.peajesencolombia.com, s. f.). En la Figura 3 se encuentra la tabla de las distancias entre ciudades.

Figura 3

Distancias entre ciudades

In [ ]:
distanciaMedida = pd.read_csv("DistanciasMedida.csv",sep=',')
distanciaMedida

Nota: Las distancias están expresadas en kilometros

4. Coste de la gasolina según la distancia: Para hallar este valor se requirieron de datos como el automóvil a elegir, el consumo por cada 100 km, la distancia entre ciudades y el precio del litro de gasolina. El automóvil seleccionado es el Renault Twingo Authentique 2010, se seleccionó este carro por su bajo consumo de gasolina. Según coches.net, el consumo de gasolina para este carro es de 3.2 L/100 Km (RENAULT TWINGO Authentique 2010 dCi 75 eco2 E5 75 cv de 2010, s. f.). El precio del litro de gasolina para el 20 de febrero del 2023 es 2,747.130 (Colombia precios de la gasolina, s. f.). En la Figura 4 se encuentra el precio total de la gasolina de los recorridos entre ciudades.

Figura 4

Costes gasolina

In [ ]:
costesGasolina = pd.read_csv("CostesGasolina.csv",sep=',')
costesGasolina

Nota: El resultado se halló gracias a la Ecuación 1

Ecuación 1

Coste total de la gasolina

$$
T = \frac{3.2 \cdot d \cdot c}{100} \cdot \frac{L}{Km}
$$
- $T = Coste \ total$
- $d = distancia$
- $c = coste \ litro \ gasolina$
- $L = litros$
- $Km = kilómetros$

Nota: Esta ecuación se utiliza para calcular el precio total de la gasolina para los recorridos entre ciudades

5. Se investiga el precio total de los peajes entre los recorridos, este dato se pudo hallar gracias a peajesencolombia.con (Edward Daniel Vidal garcia - www.peajesencolombia.com, s. f.). En la Figura 5 se visualiza la tabla del coste total de los peajes entre las ciudades.

Figura 5

Costes de los peajes

In [ ]:
costesPeajes = pd.read_csv("CostesPeajes.csv",sep=',')
costesPeajes

Nota: La página web automatizó la sumatoria de los costes de los peajes

6. Paso final: La sumatoria de los costes. Estos costes eran la suma del valor de la hora del vendedor, el coste total de los peajes y el coste total del combustible. En la Figura 6 se encuentra el coste total entre ciudades.

Figura 6

Coste total

In [ ]:
roma = pd.read_csv('Roma.csv',sep=',')
roma

Nota: Esta es la tabla que se usará para la implementación de los algoritmos.

### Utilizando el algoritmo de la colonia de hormigas

El procedimiento que se siguió para la implementación del algoritmo fue el siguiente:

1. Definición de las distancias para el problema del comerciante viajero: La tabla Coste total brinda la matriz de costes entre las ciudades del problema, se importa esta tabla para emplearla en el algoritmo.

undefined. Creación del algoritmo de la colonia de hormigas: La función creada NO es nuestra, se tomó del Notebook brindado por el profesor.

undefined. Definición de los parámetros para el algoritmo: 

       - Cantidad de hormigas: 15

       - Tasa de evaporación de las feromonas: 0.1

       - Intensificación de las feromonas para el mejor camino: 2

       - Argumentos alfa y beta: 1.0 para los dos (Tienen igual peso las feromonas y la distancia).

       - Probabilidad de elegir la mejor ruta: 0.1

4. Ejecución del algoritmo.

In [ ]:
# Segundo paso: Traer la función de ant_colony



class AntColonyOptimizer:
    def __init__(self, ants, evaporation_rate, intensification, alpha=1.0, beta=0.0, beta_evaporation_rate=0,
                 choose_best=.1):
        """
        Ant colony optimizer.  Traverses a graph and finds either the max or min distance between nodes.
        :param ants: number of ants to traverse the graph
        :param evaporation_rate: rate at which pheromone evaporates
        :param intensification: constant added to the best path
        :param alpha: weighting of pheromone
        :param beta: weighting of heuristic (1/distance)
        :param beta_evaporation_rate: rate at which beta decays (optional)
        :param choose_best: probability to choose the best route
        """
        # Parameters
        self.ants = ants
        self.evaporation_rate = evaporation_rate
        self.pheromone_intensification = intensification
        self.heuristic_alpha = alpha
        self.heuristic_beta = beta
        self.beta_evaporation_rate = beta_evaporation_rate
        self.choose_best = choose_best

        # Internal representations
        self.pheromone_matrix = None
        self.heuristic_matrix = None
        self.probability_matrix = None

        self.map = None
        self.set_of_available_nodes = None

        # Internal stats
        self.best_series = []
        self.best = None
        self.fitted = False
        self.best_path = None
        self.fit_time = None

        # Plotting values
        self.stopped_early = False

    def __str__(self):
        string = "Ant Colony Optimizer"
        string += "\n--------------------"
        string += "\nDesigned to optimize either the minimum or maximum distance between nodes in a square matrix that behaves like a distance matrix."
        string += "\n--------------------"
        string += "\nNumber of ants:\t\t\t\t{}".format(self.ants)
        string += "\nEvaporation rate:\t\t\t{}".format(self.evaporation_rate)
        string += "\nIntensification factor:\t\t{}".format(self.pheromone_intensification)
        string += "\nAlpha Heuristic:\t\t\t{}".format(self.heuristic_alpha)
        string += "\nBeta Heuristic:\t\t\t\t{}".format(self.heuristic_beta)
        string += "\nBeta Evaporation Rate:\t\t{}".format(self.beta_evaporation_rate)
        string += "\nChoose Best Percentage:\t\t{}".format(self.choose_best)
        string += "\n--------------------"
        string += "\nUSAGE:"
        string += "\nNumber of ants influences how many paths are explored each iteration."
        string += "\nThe alpha and beta heuristics affect how much influence the pheromones or the distance heuristic weigh an ants' decisions."
        string += "\nBeta evaporation reduces the influence of the heuristic over time."
        string += "\nChoose best is a percentage of how often an ant will choose the best route over probabilistically choosing a route based on pheromones."
        string += "\n--------------------"
        if self.fitted:
            string += "\n\nThis optimizer has been fitted."
        else:
            string += "\n\nThis optimizer has NOT been fitted."
        return string

    def _initialize(self):
        """
        Initializes the model by creating the various matrices and generating the list of available nodes
        """
        assert self.map.shape[0] == self.map.shape[1], "Map is not a distance matrix!"
        num_nodes = self.map.shape[0]
        self.pheromone_matrix = np.ones((num_nodes, num_nodes))
        # Remove the diagonal since there is no pheromone from node i to itself
        self.pheromone_matrix[np.eye(num_nodes) == 1] = 0
        self.heuristic_matrix = 1 / self.map
        self.probability_matrix = (self.pheromone_matrix ** self.heuristic_alpha) * (
                self.heuristic_matrix ** self.heuristic_beta)  # element by element multiplcation
        self.set_of_available_nodes = list(range(num_nodes))

    def _reinstate_nodes(self):
        """
        Resets available nodes to all nodes for the next iteration
        """
        self.set_of_available_nodes = list(range(self.map.shape[0]))

    def _update_probabilities(self):
        """
        After evaporation and intensification, the probability matrix needs to be updated.  This function
        does that.
        """
        self.probability_matrix = (self.pheromone_matrix ** self.heuristic_alpha) * (
                self.heuristic_matrix ** self.heuristic_beta)

    def _choose_next_node(self, from_node):
        """
        Chooses the next node based on probabilities.  If p < p_choose_best, then the best path is chosen, otherwise
        it is selected from a probability distribution weighted by the pheromone.
        :param from_node: the node the ant is coming from
        :return: index of the node the ant is going to
        """
        numerator = self.probability_matrix[from_node, self.set_of_available_nodes]
        if np.random.random() < self.choose_best:
            next_node = np.argmax(numerator)
        else:
            denominator = np.sum(numerator)
            probabilities = numerator / denominator
            next_node = np.random.choice(range(len(probabilities)), p=probabilities)
        return next_node

    def _remove_node(self, node):
        self.set_of_available_nodes.remove(node)

    def _evaluate(self, paths, mode):
        """
        Evaluates the solutions of the ants by adding up the distances between nodes.
        :param paths: solutions from the ants
        :param mode: max or min
        :return: x and y coordinates of the best path as a tuple, the best path, and the best score
        """
        scores = np.zeros(len(paths))
        coordinates_i = []
        coordinates_j = []
        for index, path in enumerate(paths):
            score = 0
            coords_i = []
            coords_j = []
            for i in range(len(path) - 1):
                coords_i.append(path[i])
                coords_j.append(path[i + 1])
                score += self.map[path[i], path[i + 1]]
            scores[index] = score
            coordinates_i.append(coords_i)
            coordinates_j.append(coords_j)
        if mode == 'min':
            best = np.argmin(scores)
        elif mode == 'max':
            best = np.argmax(scores)
        return (coordinates_i[best], coordinates_j[best]), paths[best], scores[best]

    def _evaporation(self):
        """
        Evaporate some pheromone as the inverse of the evaporation rate.  Also evaporates beta if desired.
        """
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        self.heuristic_beta *= (1 - self.beta_evaporation_rate)

    def _intensify(self, best_coords):
        """
        Increases the pheromone by some scalar for the best route.
        :param best_coords: x and y (i and j) coordinates of the best route
        """
        i = best_coords[0]
        j = best_coords[1]
        self.pheromone_matrix[i, j] += self.pheromone_intensification

    def fit(self, map_matrix, iterations=100, mode='min', early_stopping_count=20, verbose=True):
        """
        Fits the ACO to a specific map.  This was designed with the Traveling Salesman problem in mind.
        :param map_matrix: Distance matrix or some other matrix with similar properties
        :param iterations: number of iterations
        :param mode: whether to get the minimum path or maximum path
        :param early_stopping_count: how many iterations of the same score to make the algorithm stop early
        :return: the best score
        """
        if verbose: print("Beginning ACO Optimization with {} iterations...".format(iterations))
        self.map = map_matrix
        start = time.time()
        self._initialize()
        num_equal = 0

        for i in range(iterations):
            start_iter = time.time()
            paths = []
            path = []

            for ant in range(self.ants):
                current_node = self.set_of_available_nodes[np.random.randint(0, len(self.set_of_available_nodes))]
                start_node = current_node
                while True:
                    path.append(current_node)
                    self._remove_node(current_node)
                    if len(self.set_of_available_nodes) != 0:
                        current_node_index = self._choose_next_node(current_node)
                        current_node = self.set_of_available_nodes[current_node_index]
                    else:
                        break

                path.append(start_node)  # go back to start
                self._reinstate_nodes()
                paths.append(path)
                path = []

            best_path_coords, best_path, best_score = self._evaluate(paths, mode)

            if i == 0:
                best_score_so_far = best_score
            else:
                if mode == 'min':
                    if best_score < best_score_so_far:
                        best_score_so_far = best_score
                        self.best_path = best_path
                elif mode == 'max':
                    if best_score > best_score_so_far:
                        best_score_so_far = best_score
                        self.best_path = best_path

            if best_score == best_score_so_far:
                num_equal += 1
            else:
                num_equal = 0

            self.best_series.append(best_score)
            self._evaporation()
            self._intensify(best_path_coords)
            self._update_probabilities()

            if verbose: print("Best score at iteration {}: {}; overall: {} ({}s)"
                              "".format(i, round(best_score, 2), round(best_score_so_far, 2),
                                        round(time.time() - start_iter)))

            if best_score == best_score_so_far and num_equal == early_stopping_count:
                self.stopped_early = True
                print("Stopping early due to {} iterations of the same score.".format(early_stopping_count))
                break

        self.fit_time = round(time.time() - start)
        self.fitted = True

        if mode == 'min':
            self.best = self.best_series[np.argmin(self.best_series)]
            if verbose: print(
                "ACO fitted.  Runtime: {} minutes.  Best score: {}".format(self.fit_time // 60, self.best))
            return self.best
        elif mode == 'max':
            self.best = self.best_series[np.argmax(self.best_series)]
            if verbose: print(
                "ACO fitted.  Runtime: {} minutes.  Best score: {}".format(self.fit_time // 60, self.best))
            return self.best
        else:
            raise ValueError("Invalid mode!  Choose 'min' or 'max'.")

    def plot(self):
        """
        Plots the score over time after the model has been fitted.
        :return: None if the model isn't fitted yet
        """
        if not self.fitted:
            print("Ant Colony Optimizer not fitted!  There exists nothing to plot.")
            return None
        else:
            fig, ax = plt.subplots(figsize=(20, 15))
            ax.plot(self.best_series, label="Best Run")
            ax.set_xlabel("Iteration")
            ax.set_ylabel("Performance")
            ax.text(.8, .6,
                    'Ants: {}\nEvap Rate: {}\nIntensify: {}\nAlpha: {}\nBeta: {}\nBeta Evap: {}\nChoose Best: {}\n\nFit Time: {}m{}'.format(
                        self.ants, self.evaporation_rate, self.pheromone_intensification, self.heuristic_alpha,
                        self.heuristic_beta, self.beta_evaporation_rate, self.choose_best, self.fit_time // 60,
                        ["\nStopped Early!" if self.stopped_early else ""][0]),
                    bbox={'facecolor': 'gray', 'alpha': 0.8, 'pad': 10}, transform=ax.transAxes)
            ax.legend()
            plt.title("Ant Colony Optimization Results (best: {})".format(np.round(self.best, 2)))
            plt.show()

In [ ]:
# Tercero: Importar nuestra información de costes entre ciudades
costeCiudades = pd.read_csv("Roma.csv",sep=',')
# Obtenemos los nombres de las ciudades
ciudades = costeCiudades.columns[1:]
# Ahora, ¿Cuál es la idea? Pasar el .csv a una lista de listas (matriz) que muestre los costes entre las ciudades
costeMatriz = costeCiudades.values.tolist()
costeMatriz = [x[1:] for x in costeMatriz]
# Ahora falta convertir la lista de listas a una lista de listas de numpy
costeMatriz = np.array(costeMatriz)


In [ ]:
# Ahora la idea es utilizar el algoritmo de colonia de hormigas
optimizador = AntColonyOptimizer(ants=15, evaporation_rate=.1, intensification=2, alpha=1, beta=1,
                               beta_evaporation_rate=0, choose_best=.1)
solucion = optimizador.fit(costeMatriz, 100)


La Figura 7 representa la evolución del algoritmo por cada iteración que pasa, llegando a su optimo desde la iteración 22.

Figura 7

Resultados de la optimización (Colonia de hormigas)

In [ ]:
optimizador.plot()

Nota: El objetivo de la optimización es obtener el menor puntaje posible por parte del Performance (El menor coste posible).

Con el resultado anterior se halla el orden de las ciudades optimo según el algoritmo de colonia de hormigas.

In [ ]:
# Ahora vamos a ver cómo es el camino que se definió
indicesOrden = optimizador.best_path
ciudadesOrden = [ciudades[x] for x in indicesOrden]
print("El orden de las ciudades es:\n"+' -> '.join(ciudadesOrden))

En la Animación 1 se puede ver gráficamente el resultado del orden de recorrido entre las ciudades

Animación 1

Recorrido entre ciudades (Algoritmo colonia de hormigas)

In [ ]:
from IPython.display import Video

Video("VideoDosUno.mp4",embed=True,width=800,height=600)

Nota: El vídeo se pudo realizar gracias a la plataforma gratuita mult.

### Utilizando algoritmos genéticos

El procedimiento que se siguió para la implementación del algoritmo fue el siguiente:

1. Definición de las distancias para el problema del comerciante viajero: La tabla Coste total brinda la matriz de costes entre las ciudades del problema, se importa esta tabla para emplearla en el algoritmo.

undefined. Creación del algoritmo genético: Se utiliza la librería Pygad para el uso del algoritmo.

undefined. Definición de los parámetros para el algoritmo: 

       - Número de generaciones: 100

       - Número de soluciones seleccionadas como padres: 15.

       - Soluciones por población: 50

       - Porcentaje de genes con mutación: 0.1

       - Tipo de selección para la definición de los padres: Rank (Los mejores según la función fitness)

       - Tipo de cruzamiento: Single point.

       - Tipo de mutación: Aleatoria.

       - Se mantienen todos los padres en la próxima generación.

       - No se permiten genes duplicados (La idea es recorrer las ciudades sin repetición).

4. Creación de la función fitness: El objetivo de la solución es minimizar los costes de los recorridos entre ciudades, entonces la función fitness resta el coste del recorrido (El total de este es negativo). Al final se elige la solución que tenga el mayor valor fitness.

4. Ejecución del algoritmo.

In [ ]:
# Instalación de las librerias
!pip3 install pygad

In [ ]:
# Import de la libreria
import pygad

In [ ]:
def fitness_func(solution, solution_idx):
    fitness = 0.0
    for i in range(len(solution)):
        if i == len(solution)-1:
            j = 0
        else:
            j = i+1
        fitness -= costeMatriz[int(solution[i]), int(solution[j])]
    return fitness

In [ ]:
num_generations = 100
num_parents_mating = 15
sol_per_pop = 50
mutation_percent_genes = 10
parent_selection_type = "rank"
crossover_type =  "single_point"
mutation_type = "random"
keep_parents = -1

In [ ]:
num_genes = len(ciudades)
gene_space = list(range(num_genes))

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       gene_space=gene_space,
                       mutation_percent_genes=mutation_percent_genes,
                       parent_selection_type=parent_selection_type,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       keep_parents=keep_parents,
                       allow_duplicate_genes=False,
                       random_seed=10,
                       save_best_solutions=True)

ga_instance.run()

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Luego del procedimiento se arroja el siguiente resultado

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
ciudadesOrden2 = [ciudades[int(x)] for x in solution]
print("El orden de las ciudades es:\n"+' -> '.join(ciudadesOrden2))

En la Animación 2 se puede ver gráficamente el resultado del orden de recorrido entre las ciudades

Animación 2

Recorrido entre ciudades (Algoritmo genético)

In [ ]:
from IPython.display import Video

Video("VideoDosDos.mp4",embed=True,width=800,height=600)

Nota: El vídeo se pudo realizar gracias a la plataforma gratuita mult.

### Conclusiones generales

Aunque los dos algoritmos tengan resultados diferentes (rutas diferentes) esto no significa que tengan cierta semejanza con respecto al orden y relaciones entre ciudades, es decir, que existen algunos caminos en común. Una diferencia importante del por qué de este resultado es la necesidad del algoritmo de colonias de hormigas de volver al municipio de donde se empezó, algo que no es necesario para el algoritmo genético. Otro punto importante a resaltar es la sensibilidad de los resultados a los cambios de los números aleatorios o a los parámetros del algoritmo, un buen resultado es dependiente de los parámetros que haya definido el programador.

### Referencias

- Wikipedia contributors. (2023, 20 enero). Rastrigin function. Wikipedia. https://en.wikipedia.org/wiki/Rastrigin_function

- Sonja Surjanovic & Derek Bingham, Simon Fraser University. (2023, 20 enero). https://www.sfu.ca/%7Essurjano/schwef.html

- pygad Module — PyGAD 2.19.2 documentation. (s/f). Readthedocs.Io. Recuperado el 5 de marzo de 2023, de https://pygad.readthedocs.io/en/latest/README_pygad_ReadTheDocs.html

- Rome2rio: descubre cómo llegar a cualquier lugar. (s. f.). Rome2rio. Recuperado 23 de febrero de 2023, de https://www.rome2rio.com/es/

- Alvarez, M. G. (2022, 18 diciembre). Cómo quedaron las horas extra, nocturnas y festivos con mínimo de 2023. Portafolio.co. Recuperado 23 de febrero de 2023, de https://www.portafolio.co/economia/finanzas/salario-minimo-2023-asi-quedaron-horas-extra-nocturnas-y-festivos-575787

- RENAULT TWINGO Authentique 2010 dCi 75 eco2 E5 75 cv de 2010. (s. f.). cochesnet. Recuperado 23 de febrero de 2023, de https://www.coches.net/fichas_tecnicas/renault/twingo/berlina/3-puertas/authentique_2010_dci_75_eco2_e5_75cv_diesel/43398/752945620101001/

- Colombia precios de la gasolina. (s. f.). GlobalPetrolPrices.com. Recuperado 23 de febrero de 2023, de https://es.globalpetrolprices.com/Colombia/gasoline_prices/

- Edward Daniel Vidal garcia - www.peajesencolombia.com. (s. f.). Peajes en Colombia y tarifas de los peajes en el 2022. Peajes en Colombia. Recuperado 23 de febrero de 2023, de https://www.peajesencolombia.com/

- SciPy v1.10.1 Manual. (s. f.). SciPy documentation. Recuperado 10 de marzo de 2023, de https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html

- Particle Swarm Optimization Algorithm. (s. f.). notebook.community. Recuperado 10 de marzo de 2023, de https://notebook.community/CAChemE/stochastic-optimization/PSO/1D/1D-Python-PSO-algorithm-viz

- Wikipedia contributors. (2023, 27 febrero). Particle swarm optimization. Wikipedia. Recuperado 10 de marzo de 2023, de https://en.wikipedia.org/wiki/Particle_swarm_optimization

- Differential Evolution from Scratch in Python. (2021, 16 junio). Machine Learning Mastery. Recuperado 10 de marzo de 2023, de https://machinelearningmastery.com/differential-evolution-from-scratch-in-python/

- Wikipedia contributors. (2022, 3 noviembre). Differential evolution. Wikipedia. Recuperado 10 de marzo de 2023, de https://en.wikipedia.org/wiki/Differential_evolution

- Notebook.community. (n.d.-b). https://notebook.community/CAChemE/stochastic-optimization/PSO/2D/2D-Python-PSO-algorithm-viz

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=66255e75-fd2c-4974-aeca-21e74e31f9ba' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>